# Text Generation with RNNs

**by Bobby Cheng**

Given a sequence of characters from the Shakespear data, we will train a model to predict the next most likely character in the given sequence.


## Setup

### Import Libraries

In [1]:
## Import Libraries

import tensorflow as tf
import numpy as np
import os
import time

### Download and Read the Dataset

In [94]:
path_to_file = tf.keras.utils.get_file('bible.txt', 'https://raw.githubusercontent.com/mxw/grmr/master/src/finaltests/bible.txt')

# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')


Length of text: 4451368 characters


In [3]:
# Take a look at the first 250 characters in text
print(text[:250])

1:1 In the beginning God created the heaven and the earth.

1:2 And the earth was without form, and void; and darkness was upon
the face of the deep. And the Spirit of God moved upon the face of the
waters.

1:3 And God said, Let there be light


In [4]:
# The number of unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

81 unique characters


## Process the Text
### Vectorize the Text

In [5]:
example_texts = ['abcde', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e'], [b'x', b'y', b'z']]>

In [6]:
## Create a function that will convert a string of characters into numerical IDs
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

# illustrate how ids_from_chars will convert 'chars' into integers
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[56, 57, 58, 59, 60], [79, 80, 81]]>

In [7]:
## Write a function that will convert the string of numerical IDs into characters
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e'], [b'x', b'y', b'z']]>

In [8]:
## with the above reinstated chars value, we can reconvert it back to a single string
tf.strings.reduce_join(chars, axis=-1).numpy()

## Note that 'chars' is a tensor object. Hence, it cannot be joined back with ''.join(<list>).

array([b'abcde', b'xyz'], dtype=object)

In [9]:
## we can create a useful fuinction that would form tensor integers back into words
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

text_from_ids(ids)

<tf.Tensor: shape=(2,), dtype=string, numpy=array([b'abcde', b'xyz'], dtype=object)>

### Create Training Examples and Targets

In [10]:
## Let us now convert the text data into ids!
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
print(len(all_ids))

## this is the same length of words in the text

4451368


In [11]:
## We'll now create a dataset tensor object that will aid us in our text generation operations
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

## The following will illustrate what the dataset tensor object can do.
for ids in ids_dataset.take(11):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

1
:
1
 
I
n
 
t
h
e
 


In [12]:
## We'll see how useful the dataset tensor object is.
## in this step, we'll create a sequences object from the dataset object. 
## Observe how the sequences object now behaves a bit like a 'batch' logic
seq_length = 100 # feel free to vary this integer variable
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(5): # feel free to vary this integer variable
  print(seq)
  print('observe the conversion from ids to text:')
  print(text_from_ids(seq).numpy())
  print("\n")

tf.Tensor(
[17 26 17  3 38 69  3 75 63 60  3 57 60 62 64 69 69 64 69 62  3 36 70 59
  3 58 73 60 56 75 60 59  3 75 63 60  3 63 60 56 77 60 69  3 56 69 59  3
 75 63 60  3 60 56 73 75 63 14  2  1  2  1 17 26 18  3 30 69 59  3 75 63
 60  3 60 56 73 75 63  3 78 56 74  3 78 64 75 63 70 76 75  3 61 70 73 68
 12  3 56 69 59], shape=(101,), dtype=int64)
observe the conversion from ids to text:
b'1:1 In the beginning God created the heaven and the earth.\r\n\r\n1:2 And the earth was without form, and'


tf.Tensor(
[ 3 77 70 64 59 27  3 56 69 59  3 59 56 73 66 69 60 74 74  3 78 56 74  3
 76 71 70 69  2  1 75 63 60  3 61 56 58 60  3 70 61  3 75 63 60  3 59 60
 60 71 14  3 30 69 59  3 75 63 60  3 48 71 64 73 64 75  3 70 61  3 36 70
 59  3 68 70 77 60 59  3 76 71 70 69  3 75 63 60  3 61 56 58 60  3 70 61
  3 75 63 60  2], shape=(101,), dtype=int64)
observe the conversion from ids to text:
b' void; and darkness was upon\r\nthe face of the deep. And the Spirit of God moved upon the face of the\r'


t

In [13]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

dataset_subclassing = sequences.map(split_input_target)

for input_example, target_example in dataset_subclassing.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'1:1 In the beginning God created the heaven and the earth.\r\n\r\n1:2 And the earth was without form, an'
Target: b':1 In the beginning God created the heaven and the earth.\r\n\r\n1:2 And the earth was without form, and'


### Create Training Batches

In [14]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset_subclassing = (
    dataset_subclassing
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset_subclassing

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

### Building the Model

In [15]:
vocab_size = len(ids_from_chars.get_vocabulary())
embedding_dim = 256
rnn_units = 1024

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Embedding

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = Sequential()
    model.add(Embedding(vocab_size
                        ,embedding_dim
                        ,batch_input_shape=(batch_size,None)))
    model.add(GRU(rnn_units
                  ,return_sequences=True
                  ,stateful=True))
    model.add(Dense(vocab_size))

    return model

In [18]:
model = build_model(vocab_size = vocab_size
                       ,embedding_dim=embedding_dim
                       ,rnn_units=rnn_units
                       ,batch_size=BATCH_SIZE)

In [19]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (64, None, 256)           20992     
                                                                 
 gru_1 (GRU)                 (64, None, 1024)          3938304   
                                                                 
 dense_1 (Dense)             (64, None, 82)            84050     
                                                                 
Total params: 4,043,346
Trainable params: 4,043,346
Non-trainable params: 0
_________________________________________________________________


In [20]:
for input_example_batch, target_example_batch in dataset_subclassing.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 82) # (batch_size, sequence_length, vocab_size)


In [21]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices

array([ 0, 50, 71, 20, 70, 55, 11, 32, 67, 58, 37, 76, 70, 76,  4, 67, 14,
       19, 49, 72, 52, 61, 23, 64, 41, 18, 16, 74, 13, 35, 58, 21, 78,  7,
       27, 54, 23, 47,  5, 17, 70,  7, 25, 61, 33,  3, 53, 42,  9, 19, 34,
       46, 54, 34, 61, 50, 77, 58, 13, 17, 73,  9, 34, 75, 65, 12, 61,  4,
        2, 19, 78,  4, 40, 72, 52, 19, 37, 14, 13, 75, 28, 61, 60,  7, 72,
        1, 29, 32, 73, 44,  2, 51, 15, 12, 43, 30, 12, 79, 41, 77])

### Train the Model

In [22]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 82)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.4062257


In [23]:
model.compile(optimizer='adam', loss=loss)

In [24]:
EPOCHS=20
history = model.fit(dataset_subclassing, epochs=EPOCHS)#, callbacks=[checkpoint_callback])

Epoch 1/20
688/688 [==============================] - 2109s 3s/step - loss: 1.7533
Epoch 2/20
688/688 [==============================] - 3030s 4s/step - loss: 1.1778
Epoch 3/20
688/688 [==============================] - 2841s 4s/step - loss: 1.0850
Epoch 4/20
688/688 [==============================] - 4240s 6s/step - loss: 1.0392
Epoch 5/20
688/688 [==============================] - 2553s 4s/step - loss: 1.0081
Epoch 6/20
688/688 [==============================] - 1470s 2s/step - loss: 0.9836
Epoch 7/20
688/688 [==============================] - 1433s 2s/step - loss: 0.9639
Epoch 8/20
688/688 [==============================] - 2031s 3s/step - loss: 0.9466
Epoch 9/20
688/688 [==============================] - 2386s 3s/step - loss: 0.9323
Epoch 10/20
688/688 [==============================] - 1660s 2s/step - loss: 0.9200
Epoch 11/20
688/688 [==============================] - 1462s 2s/step - loss: 0.9093
Epoch 12/20
688/688 [==============================] - 1339s 2s/step - loss: 0.9004
E

In [25]:
model.save_weights('sequential-bible-weights.h5')

### Generate Text

In [26]:
bible_text_generator = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

bible_text_generator.load_weights('sequential-bible-weights.h5')

bible_text_generator.build(tf.TensorShape([1, None]))

In [27]:
bible_text_generator.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (1, None, 256)            20992     
                                                                 
 gru_2 (GRU)                 (1, None, 1024)           3938304   
                                                                 
 dense_2 (Dense)             (1, None, 82)             84050     
                                                                 
Total params: 4,043,346
Trainable params: 4,043,346
Non-trainable params: 0
_________________________________________________________________


In [30]:
def generate_text(model, start_string, temperature = 1.0, prediction_length = 1000):  
  
    num_generate = prediction_length
    input_eval = [ids_from_chars(s) for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
   
    text_generated = []

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(chars_from_ids(predicted_id))

    return (start_string + tf.strings.join(text_generated).numpy().decode('utf-8'))

In [36]:
print(generate_text(bible_text_generator, start_string=u"Jesus", temperature = 0.8, prediction_length = 500))

Jesus said unto him, All the
people which ye had read throughout Pilate that believed not: but if
it be for my members shall the flesh of an only persecutions.

1:13 How long shall I obtailedo him?  17:12 And when the other sea: and
they shall be absent because of the LORD's house.

4:12 Behold, I will send a fire unto the body to concerning him that placeth, where
are the children of the flesh, that ye be not seen situate by faith unto the commandment
of God, and from the strange Lord Jesus


In [118]:
print(generate_text(bible_text_generator, start_string=u"Love the Lord with all your ", temperature = 0.2, prediction_length = 500))

Love the Lord with all your hearts:
for they are many that were not ashamed of the faith of the LORD, and the strangers shall be as the
same sacrifice was just, and the stars of heaven stood by the coast of the earth, and the hair of his hands toward the
south.

13:17 And he said unto them, Who are the seven lambs of the first year of king Cyrus, and the
children of Israel, and the first and the scribes, and the
Levites, and the Jebusites, and the Jebusites, and the
Gentiles, and the sons of Zebedee, and of the chi


In [13]:
## Becuase we are going to use the sequential API to run our predictions,
## let's try to recall what our model is trying to do.

## X inputs will be an array of integers
## But because the model makes predictions with softmax, the y needs to be one hot encoded.
## Hence, y will be one hot encoded arrays with zeros and one 1.as_integer_ratio

def split_input_target_sequential(sequence, vocab_len = len(vocab)+1):
    input_text = sequence[:-1]
    target_text = tf.one_hot(sequence[-1:], vocab_len)[0]
    
    return input_text, target_text

# split_input_target_sequential(list("Tensorflow"))
split_input_target_sequential(seq, vocab_len = len(vocab))

(<tf.Tensor: shape=(100,), dtype=int64, numpy=
 array([56, 69, 59,  3, 75, 63, 60,  3, 59, 56, 73, 66, 69, 60, 74, 74,  3,
        63, 60,  3, 58, 56, 67, 67, 60, 59,  3, 43, 64, 62, 63, 75, 14,  2,
         1, 30, 69, 59,  3, 75, 63, 60,  3, 60, 77, 60, 69, 64, 69, 62,  3,
        56, 69, 59,  3, 75, 63, 60,  3, 68, 70, 73, 69, 64, 69, 62,  3, 78,
        60, 73, 60,  3, 75, 63, 60,  3, 61, 64, 73, 74, 75,  3, 59, 56, 80,
        14,  2,  1,  2,  1, 17, 26, 22,  3, 30, 69, 59,  3, 36, 70])>,
 <tf.Tensor: shape=(81,), dtype=float32, numpy=
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>)

In [14]:
a, b = split_input_target_sequential(seq)
print(a)
print(b)

tf.Tensor(
[56 69 59  3 75 63 60  3 59 56 73 66 69 60 74 74  3 63 60  3 58 56 67 67
 60 59  3 43 64 62 63 75 14  2  1 30 69 59  3 75 63 60  3 60 77 60 69 64
 69 62  3 56 69 59  3 75 63 60  3 68 70 73 69 64 69 62  3 78 60 73 60  3
 75 63 60  3 61 64 73 74 75  3 59 56 80 14  2  1  2  1 17 26 22  3 30 69
 59  3 36 70], shape=(100,), dtype=int64)
tf.Tensor(
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(82,), dtype=float32)


In [15]:
dataset_sequential = sequences.map(split_input_target_sequential)

for input_example, target_example in dataset_sequential.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids([tf.argmax(target_example, axis = -1).numpy()]).numpy())

Input : b'1:1 In the beginning God created the heaven and the earth.\r\n\r\n1:2 And the earth was without form, an'
Target: b'd'


### Create Training Batches

In [16]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset_sequential
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 82), dtype=tf.float32, name=None))>

In [17]:
for input_example, target_example in dataset.take(1):
    print(input_example[-2:])
    print(target_example[-2:])

tf.Tensor(
[[56 75  3 56 67 67  3 75 63 60  3 71 60 70 71 67 60  3 75 63 56 75  3 64
  74  3 61 70 76 69 59  3 75 63 60 73 60 64 69  2  1 74 63 56 67 67  3 57
  60  3 75 73 64 57 76 75 56 73 64 60 74  3 76 69 75 70  3 75 63 60 60 12
   3 56 69 59  3 75 63 60 80  3 74 63 56 67 67  3 74 60 73 77 60  3 75 63
  60 60 14  2]
 [75 63 56 75  3 74 64 59 60 14  2  1  2  1 18 18 26 18 21  3 30 69 59  3
  78 63 60 69  3 75 63 60  3 56 74 74  3 74 56 78  3 75 63 60  3 56 69 62
  60 67  3 70 61  3 75 63 60  3 41 44 47 33 12  3 74 63 60  3 75 63 73 76
  74 75  3 63 60 73 74 60 67 61  2  1 76 69 75 70  3 75 63 60  3 78 56 67
  67 12  3 56]], shape=(2, 100), dtype=int64)
tf.Tensor(
[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [18]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    #print("Target:", text_from_ids([tf.argmax(target_example, axis = -1).numpy()]).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : [b'e, after our likeness:\r\nand let them have dominion over the fish of the sea, and over the fowl\r\nof t'
 b' be Canaan; a servant of servants shall he be\r\nunto his brethren.\r\n\r\n9:26 And he said, Blessed be th'
 b'\n15:15 Then the dukes of Edom shall be amazed; the mighty men of Moab,\r\ntrembling shall take hold up'
 b' for our adversaries?  5:14 And he\r\nsaid, Nay; but as captain of the host of the LORD am I now come.'
 b'ives from death.\r\n\r\n2:14 And the men answered her, Our life for yours, if ye utter not\r\nthis our bus'
 b'e Belaites: of Ashbel, the family of the Ashbelites: of Ahiram,\r\nthe family of the Ahiramites: 26:39'
 b" Let my father arise, and eat of his\r\nson's venison, that thy soul may bless me.\r\n\r\n27:32 And Isaac "
 b'hen ye\r\nshall bring a sheaf of the firstfruits of your harvest unto the\r\npriest: 23:11 And he shall '
 b'\n4:48 Even those that were numbered of them, were eight thousand and\r\nfive hundred and fourscore, 4:'
 b'wash his

## Building the Model

In this section, we will use the sequential method to generate texts.

In [19]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

### Model 1 - Stateless GRU Training

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Embedding

model = Sequential()
model.add(Embedding(vocab_size, 
                    embedding_dim, input_length = 100))
model.add(GRU(rnn_units))
model.add(Dense(vocab_size, activation = 'softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 256)          20992     
                                                                 
 gru (GRU)                   (None, 1024)              3938304   
                                                                 
 dense (Dense)               (None, 82)                84050     
                                                                 
Total params: 4,043,346
Trainable params: 4,043,346
Non-trainable params: 0
_________________________________________________________________


In [20]:
model.compile(optimizer='adam',loss = 'categorical_crossentropy')#, loss=loss)

In [21]:
history = model.fit(dataset, epochs=20)#, callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 196s 1s/step - loss: 2.9270
Epoch 2/10
172/172 [==============================] - 210s 1s/step - loss: 2.3552
Epoch 3/10
172/172 [==============================] - 223s 1s/step - loss: 2.1968
Epoch 4/10
172/172 [==============================] - 235s 1s/step - loss: 2.0382
Epoch 5/10
172/172 [==============================] - 249s 1s/step - loss: 1.8616
Epoch 6/10
172/172 [==============================] - 467s 3s/step - loss: 1.6251
Epoch 7/10
172/172 [==============================] - 992s 6s/step - loss: 1.3096
Epoch 8/10
172/172 [==============================] - 714s 4s/step - loss: 0.8712
Epoch 9/10
172/172 [==============================] - 563s 3s/step - loss: 0.4325
Epoch 10/10
172/172 [==============================] - 499s 3s/step - loss: 0.1620


In [259]:
## save the model weights
model.save_weights('text-generation-gru-1.h5')


In [546]:
model.load_weights('text-generation-gru-1.h5')

In [547]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 66) # (batch_size, sequence_length, vocab_size)


#### Model Prediction

In [549]:
selected_index = 3
start = input_example_batch[selected_index]
start = np.reshape(start, (1, len(start), 1))

print('\n----------- Show initial prompt -----------')
print(tf.strings.join(chars_from_ids(input_example_batch[selected_index])).numpy().decode('utf-8'))

print('\n----------------- FITTING -----------------')
for i in range(1000):
    list_seq = start.flatten()

    start = start[:,-100:,:]
    start = np.reshape(start, (1, len(start[0]), 1))
    prediction = model.predict(start, verbose = 0)
    # index = tf.random.categorical(prediction, num_samples = 1)
    index = np.argmax(prediction)
    list_seq = np.append(list_seq, index)
    start = np.reshape(list_seq, (1, len(list_seq), 1))


print('\n----------- Show initial prompt -----------')
print(tf.strings.join(chars_from_ids(list_seq)).numpy().decode('utf-8'))


----------- Show initial prompt -----------
Conceive when after many moody thoughts
At last by notes of household harmony
They quite forget thei

----------------- FITTING -----------------

----------- Show initial prompt -----------
Conceive when after many moody thoughts
At last by notes of household harmony
They quite forget their have be buctlyst cander to

EXENEE:
O your brititiding mine,
I hoplange of y under theirious
Lind the werith y he dread frim blond freeds, well ald offir dess,
Boug ale ul our nenisired
In lose If y hadf be gree he grow he be to make the be:
And worse ve you boyy arace.
Ber would trot they lath the bee of hear me and feast to be of tourt,
Yet he bat o's lare to the groot thou lovest:
Sto good a will.
He mare to and fearself be of home to and fould not out well I conder
Or eleese very your proce to her prothereirs farefels.
That I which I do the strempertire:
Fow we have us nothing in the very troon
Marthe warrer trow in the verous of your rondseise I
And

In [551]:
selected_index = 3
start = input_example_batch[selected_index]
start = np.reshape(start, (1, len(start), 1))

print('\n----------- Show initial prompt -----------')
print(tf.strings.join(chars_from_ids(input_example_batch[selected_index])).numpy().decode('utf-8'))

print('\n----------------- fitting... -----------------')
for i in range(1000):
    list_seq = start.flatten()
    model.reset_states()
    start = start[:,-100:,:]
    start = np.reshape(start, (1, len(start[0]), 1))
    prediction = model.predict(start, verbose = 0)
    # index = tf.random.categorical(prediction, num_samples = 1)
    index = np.argmax(prediction)
    list_seq = np.append(list_seq, index)
    start = np.reshape(list_seq, (1, len(list_seq), 1))


print('\n----------- Show Predicted Sentence -----------')
print(tf.strings.join(chars_from_ids(list_seq)).numpy().decode('utf-8'))


----------- Show initial prompt -----------
Conceive when after many moody thoughts
At last by notes of household harmony
They quite forget thei

----------------- fitting... -----------------

----------- Show Predicted Sentence -----------
Conceive when after many moody thoughts
At last by notes of household harmony
They quite forget their have be buctlyst cander to

EXENEE:
O your brititiding mine,
I hoplange of y under theirious
Lind the werith y he dread frim blond freeds, well ald offir dess,
Boug ale ul our nenisired
In lose If y hadf be gree he grow he be to make the be:
And worse ve you boyy arace.
Ber would trot they lath the bee of hear me and feast to be of tourt,
Yet he bat o's lare to the groot thou lovest:
Sto good a will.
He mare to and fearself be of home to and fould not out well I conder
Or eleese very your proce to her prothereirs farefels.
That I which I do the strempertire:
Fow we have us nothing in the very troon
Marthe warrer trow in the verous of your rondseis

### Model 3 - Stateful GRU Training

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Embedding

model3 = Sequential()
model3.add(Embedding(vocab_size, 
                    embedding_dim, input_length = 100, batch_input_shape=(64, 100)))
model3.add(GRU(rnn_units, stateful = True))
model3.add(Dense(vocab_size, activation = 'softmax'))

model3.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (64, 100, 256)            20992     
                                                                 
 gru_1 (GRU)                 (64, 1024)                3938304   
                                                                 
 dense_1 (Dense)             (64, 82)                  84050     
                                                                 
Total params: 4,043,346
Trainable params: 4,043,346
Non-trainable params: 0
_________________________________________________________________


In [22]:
model3.compile(optimizer='adam',loss = 'categorical_crossentropy')#, loss=loss)

In [23]:
history3 = model3.fit(dataset, epochs=20)#, callbacks=[checkpoint_callback])

Epoch 1/20


  8/688 [..............................] - ETA: 13:18 - loss: 4.3552

KeyboardInterrupt: 

In [472]:
model3.save_weights('model3_weights.h5')

In [473]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Embedding

model5 = Sequential()
model5.add(Embedding(vocab_size, 
                    embedding_dim, input_length = 100, batch_input_shape=(1, 100)))
model5.add(GRU(rnn_units, stateful = True))
model5.add(Dense(vocab_size, activation = 'softmax'))

model5.summary()

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_18 (Embedding)    (1, 100, 256)             16896     
                                                                 
 gru_17 (GRU)                (1, 1024)                 3938304   
                                                                 
 dense_8 (Dense)             (1, 66)                   67650     
                                                                 
Total params: 4,022,850
Trainable params: 4,022,850
Non-trainable params: 0
_________________________________________________________________


In [474]:
model5.load_weights('model3_weights.h5')

In [552]:
selected_index = 3
start = input_example_batch[selected_index]
start = np.reshape(start, (1, len(start), 1))

print('\n----------- Show initial prompt -----------')
print(tf.strings.join(chars_from_ids(input_example_batch[selected_index])).numpy().decode('utf-8'))

print('\n----------------- FITTING -----------------')
for i in range(1000):
    list_seq = start.flatten()

    start = start[:,-100:,:]
    start = np.reshape(start, (1, len(start[0]), 1))
    prediction = model5.predict(start, verbose = 0)
    # index = tf.random.categorical(prediction, num_samples = 1)
    index = np.argmax(prediction)
    list_seq = np.append(list_seq, index)
    start = np.reshape(list_seq, (1, len(list_seq), 1))


print('\n----------- Show initial prompt -----------')
print(tf.strings.join(chars_from_ids(list_seq)).numpy().decode('utf-8'))


----------- Show initial prompt -----------
Conceive when after many moody thoughts
At last by notes of household harmony
They quite forget thei

----------------- FITTING -----------------

----------- Show initial prompt -----------
Conceive when after many moody thoughts
At last by notes of household harmony
They quite forget their cougn ho good wo the sweed thing howning thee the vencustoves of wind:
Thith blat make the ploud to and the ploudst on thence,
But deade to me ret me be ut the vingered they lakes bood
Which rother is are but thou lass the hender theell he freat my heart me breede the hather

of Burninnere, things she leess of Hears, theee themest thee tay
To good and not the ken of kear fould of honour of so sin,
Which rother is a mand the wholl the verious pording the wence of Whis dood with honourt to serr.

I GoLENOUS:
But deade to me reed and litie for horour wence,
Bonnearefof the wence of Whis it wed.

What'd you you to me rock at hears,
And brath rifing sween the

In [553]:
selected_index = 3
start = input_example_batch[selected_index]
start = np.reshape(start, (1, len(start), 1))

print('\n----------- Show initial prompt -----------')
print(tf.strings.join(chars_from_ids(input_example_batch[selected_index])).numpy().decode('utf-8'))

print('\n----------------- FITTING -----------------')
for i in range(1000):
    list_seq = start.flatten()
    model5.reset_states()
    start = start[:,-100:,:]
    start = np.reshape(start, (1, len(start[0]), 1))
    prediction = model5.predict(start, verbose = 0)
    # index = tf.random.categorical(prediction, num_samples = 1)
    index = np.argmax(prediction)
    list_seq = np.append(list_seq, index)
    start = np.reshape(list_seq, (1, len(list_seq), 1))


print('\n----------- Show initial prompt -----------')
print(tf.strings.join(chars_from_ids(list_seq)).numpy().decode('utf-8'))


----------- Show initial prompt -----------
Conceive when after many moody thoughts
At last by notes of household harmony
They quite forget thei

----------------- FITTING -----------------

----------- Show initial prompt -----------
Conceive when after many moody thoughts
At last by notes of household harmony
They quite forget their cougn ho good wo the sweed thing howning thee takes of wind:
Thith blat make the plopt of ham him blat I han had he hath be not the king,
Which rother it the hand of hear for the strence?

MEOEN ELIO:
As ne a mander to sitr;
Which rothering is ard net she leess of hear sleepeftertstert,
Sit thee hon she plonds the bent on thy she daudte
storm and ret my lord, I'll heme the taise:
I wnot not and let make the bears, bet exser fortinest:
Firet and string sundenting theeress she lease to theerest strinds
Which rother it your hinds she lensure of whe tause:
and ffee for her so sen sates the wence of Whis dount of for
his to and ferefole pureat,
Which rother i

### Model 0 - Using Subclassing Methods

In [26]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [27]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [28]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 82) # (batch_size, sequence_length, vocab_size)


In [29]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     multiple                  20992     
                                                                 
 gru_2 (GRU)                 multiple                  3938304   
                                                                 
 dense_2 (Dense)             multiple                  84050     
                                                                 
Total params: 4,043,346
Trainable params: 4,043,346
Non-trainable params: 0
_________________________________________________________________


In [30]:
for input_example_batch, target_example_batch in dataset_subclassing.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 82) # (batch_size, sequence_length, vocab_size)


In [31]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [32]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 82)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.4065585, shape=(), dtype=float32)


In [33]:
model.compile(optimizer='adam', loss=loss)

In [34]:
EPOCHS = 20
history = model.fit(dataset_subclassing, epochs=EPOCHS)#, callbacks=[checkpoint_callback])

Epoch 1/20


688/688 [==============================] - 1384s 2s/step - loss: 1.7402
Epoch 2/20
688/688 [==============================] - 811s 1s/step - loss: 1.1390
Epoch 3/20
688/688 [==============================] - 820s 1s/step - loss: 1.0397
Epoch 4/20
688/688 [==============================] - 957s 1s/step - loss: 0.9895
Epoch 5/20
688/688 [==============================] - 1118s 2s/step - loss: 0.9536
Epoch 6/20
688/688 [==============================] - 1678s 2s/step - loss: 0.9245
Epoch 7/20
688/688 [==============================] - 1282s 2s/step - loss: 0.8985
Epoch 8/20
688/688 [==============================] - 2405s 3s/step - loss: 0.8762
Epoch 9/20
688/688 [==============================] - 799s 1s/step - loss: 0.8557
Epoch 10/20
688/688 [==============================] - 1031s 1s/step - loss: 0.8382
Epoch 11/20
688/688 [==============================] - 2159s 3s/step - loss: 0.8222
Epoch 12/20
688/688 [==============================] - 1152s 2s/step - loss: 0.8089
Epoch 13/20
688/

In [48]:
model.save_weights('subclassing-bible-weights.h5')

In [35]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [36]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [37]:
start = time.time()
states = None
next_char = tf.constant(['1:1'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

1:19 And when I cried with you for us, that we may enlighteous
alone, having hope in the kingdom of our Lord, Pray ye thou virgin a
place of things unto him trouble.

14:31 Greet Baphaba the Beronian, that instead of the Procect
of the Project Gutenberg Literary Artole from the east side on the thirty and
sixteen thousand.

22:41 And they withdred the palt of the hands, and lew on him, and passed toward
Sephzarim.

12:14 And he would have made the first that labouring at home, having roof to
cast them out of my sight, and say, Hear ye the words
of a loud voice, As the LORD commanded Abrahmus are a council, ye shall
hear it, nor son have not received sight.

5:11 I cried with unto me, saith the LORD.

23:84 Behold, I count them, the same merciful, stand in the grave toward this
people, and his locks, born in the cities that are first.

47:20 The stone most high processeth son; and these are her father
with you but by dyitter a godly seed, bringing instead of a free
g 

_________________

In [3]:
start = time.time()
states = None
next_char = tf.constant(['Jesus'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

NameError: name 'one_step_model' is not defined

## Ideal Model Building

In [37]:
# evaluation using BLEU
from nltk.translate.bleu_score import corpus_bleu


In [39]:
pred = generate_text(bible_text_generator, start_string=u"Jesus", temperature = 0.8, prediction_length = 500)
print(pred)

Jesus
Christ is the spirit of the LORD. Now there eat the water of the sea;
in silver and gold, and in the world by the way, through him that believeth in the
ends of the earth, and not that we should bear the pen your souls.

1:9 For what will I bring again the chief priests for the increase of the General that told him, saying, Abide in the house of
Israel, that he saith unto the servants the chief caperia, and all the
children of Israel that bare not the holy mouth of the world, and it
sha


In [43]:
def generate_text_mod(model, start_string, temperature = 1.0, prediction_length = 1000):  
  
    num_generate = prediction_length
    input_eval = [ids_from_chars(s) for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
   
    text_generated = []

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(chars_from_ids(predicted_id))

    return text_generated

In [44]:
pred = generate_text_mod(bible_text_generator, start_string=u"Jesus", temperature = 0.8, prediction_length = 500)
pred

[<tf.Tensor: shape=(), dtype=string, numpy=b','>,
 <tf.Tensor: shape=(), dtype=string, numpy=b' '>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'w'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'h'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'o'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b' '>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'd'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'i'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'd'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b' '>,
 <tf.Tensor: shape=(), dtype=string, numpy=b't'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'h'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'e'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'y'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b' '>,
 <tf.Tensor: shape=(), dtype=string, numpy=b't'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'h'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'i'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'n'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'k'>,


In [76]:
import re

text = tf.strings.join(pred).numpy()
re.split(b',|\n|\r| ', text)


[b'',
 b'',
 b'who',
 b'did',
 b'they',
 b'think',
 b'their',
 b'',
 b'covenant',
 b'',
 b'and',
 b'our',
 b'transgressions',
 b'among',
 b'the',
 b'world',
 b'',
 b'even',
 b'',
 b'as',
 b'this',
 b'day.',
 b'',
 b'',
 b'',
 b'22:22',
 b'And',
 b'he',
 b'said',
 b'unto',
 b'them',
 b'',
 b'Matten',
 b'seek',
 b'hold',
 b'the',
 b'people',
 b'of',
 b'the',
 b'land.',
 b'',
 b'',
 b'',
 b'3:5',
 b'Wilt',
 b'thou',
 b'join',
 b'the',
 b'light',
 b'of',
 b'my',
 b'hand!',
 b'',
 b'24:22',
 b'Thou',
 b'hast',
 b'already',
 b'to',
 b'be',
 b'brought',
 b'forth',
 b'',
 b'and',
 b'if',
 b'ye',
 b'will',
 b'be',
 b'no',
 b'',
 b'bread',
 b'for',
 b'the',
 b'commandment',
 b'of',
 b'God',
 b'',
 b'and',
 b'then',
 b'shall',
 b'he',
 b'eat',
 b'the',
 b'',
 b'burnt',
 b'offering',
 b'of',
 b'a',
 b'Prine',
 b'of',
 b'great',
 b'power;',
 b'1:5',
 b'To',
 b'the',
 b'house',
 b'of',
 b'Israel',
 b'',
 b'must',
 b'need',
 b'with',
 b'a',
 b'great',
 b'stead',
 b'of',
 b'a',
 b'burnt',
 b'dead',
 b

In [109]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')
candidates = tokenizer.tokenize(tf.strings.join(pred).numpy().decode('utf-8'))
candidates

['who',
 'did',
 'they',
 'think',
 'their',
 'covenant',
 'and',
 'our',
 'transgressions',
 'among',
 'the',
 'world',
 'even',
 'as',
 'this',
 'day',
 '22',
 '22',
 'And',
 'he',
 'said',
 'unto',
 'them',
 'Matten',
 'seek',
 'hold',
 'the',
 'people',
 'of',
 'the',
 'land',
 '3',
 '5',
 'Wilt',
 'thou',
 'join',
 'the',
 'light',
 'of',
 'my',
 'hand',
 '24',
 '22',
 'Thou',
 'hast',
 'already',
 'to',
 'be',
 'brought',
 'forth',
 'and',
 'if',
 'ye',
 'will',
 'be',
 'no',
 'bread',
 'for',
 'the',
 'commandment',
 'of',
 'God',
 'and',
 'then',
 'shall',
 'he',
 'eat',
 'the',
 'burnt',
 'offering',
 'of',
 'a',
 'Prine',
 'of',
 'great',
 'power',
 '1',
 '5',
 'To',
 'the',
 'house',
 'of',
 'Israel',
 'must',
 'need',
 'with',
 'a',
 'great',
 'stead',
 'of',
 'a',
 'burnt',
 'dead',
 'fell',
 'down',
 'at',
 'the',
 'first',
 'set',
 'on',
 'the',
 'one',
 'wa']

In [98]:

references = [tokenizer.tokenize(text)]
references


[['1',
  '1',
  'In',
  'the',
  'beginning',
  'God',
  'created',
  'the',
  'heaven',
  'and',
  'the',
  'earth',
  '1',
  '2',
  'And',
  'the',
  'earth',
  'was',
  'without',
  'form',
  'and',
  'void',
  'and',
  'darkness',
  'was',
  'upon',
  'the',
  'face',
  'of',
  'the',
  'deep',
  'And',
  'the',
  'Spirit',
  'of',
  'God',
  'moved',
  'upon',
  'the',
  'face',
  'of',
  'the',
  'waters',
  '1',
  '3',
  'And',
  'God',
  'said',
  'Let',
  'there',
  'be',
  'light',
  'and',
  'there',
  'was',
  'light',
  '1',
  '4',
  'And',
  'God',
  'saw',
  'the',
  'light',
  'that',
  'it',
  'was',
  'good',
  'and',
  'God',
  'divided',
  'the',
  'light',
  'from',
  'the',
  'darkness',
  '1',
  '5',
  'And',
  'God',
  'called',
  'the',
  'light',
  'Day',
  'and',
  'the',
  'darkness',
  'he',
  'called',
  'Night',
  'And',
  'the',
  'evening',
  'and',
  'the',
  'morning',
  'were',
  'the',
  'first',
  'day',
  '1',
  '6',
  'And',
  'God',
  'said',
  

In [111]:
from nltk.translate.bleu_score import sentence_bleu

score = sentence_bleu([list(set(tokenizer.tokenize(text)))], candidates)
print(score)

7.431554310846816e-291


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram o

In [107]:
list(set(tokenizer.tokenize(text)))

['Perazim',
 'Betharabah',
 'chop',
 'Libni',
 'inheritor',
 'unstable',
 'desolation',
 'clothes',
 'almond',
 'Urijah',
 '171',
 'disposing',
 'spindle',
 'Joiarib',
 'Spring',
 'Arba',
 'temper',
 'row',
 'accompanying',
 'desiredst',
 'rings',
 'Should',
 'seats',
 'hart',
 'Booz',
 'within',
 'Tabbath',
 'whirlwind',
 '91',
 'Sargon',
 'gutenberg',
 'runneth',
 'Tekoa',
 'spoilest',
 'Beloved',
 'Naarath',
 'gold',
 'gave',
 'TRADEMARK',
 'accursed',
 'overthrow',
 'Gilboa',
 'Hannathon',
 'Remain',
 'Rohgah',
 'ghost',
 'bemoan',
 'Irshemesh',
 'Kedar',
 'tingle',
 'odd',
 'shameful',
 'can',
 'Geshur',
 'shamefacedness',
 'Eliphaz',
 'enjoin',
 'company',
 'washing',
 'poorer',
 'hasty',
 'Zephon',
 'Meshullemeth',
 'gender',
 'Gemariah',
 'couches',
 'spared',
 'possessions',
 'Tilgathpilneser',
 'deceiveth',
 'Havilah',
 'Forasmuch',
 'gropeth',
 'Pharisees',
 'transforming',
 'Happy',
 'Withal',
 'Cherith',
 'wagon',
 'Hazaraddar',
 'Obil',
 'Carcas',
 'warning',
 'buriers',


In [31]:
tf.random.categorical(example_batch_predictions, num_samples = 1)

<tf.Tensor: shape=(64, 1), dtype=int64, numpy=
array([[10],
       [44],
       [46],
       [49],
       [51],
       [24],
       [15],
       [33],
       [13],
       [19],
       [52],
       [21],
       [41],
       [52],
       [35],
       [50],
       [21],
       [44],
       [ 8],
       [15],
       [37],
       [ 2],
       [47],
       [61],
       [64],
       [13],
       [47],
       [ 2],
       [42],
       [ 8],
       [54],
       [60],
       [51],
       [18],
       [50],
       [45],
       [33],
       [31],
       [18],
       [44],
       [25],
       [11],
       [ 7],
       [59],
       [25],
       [40],
       [48],
       [16],
       [45],
       [56],
       [23],
       [ 5],
       [17],
       [60],
       [18],
       [ 2],
       [23],
       [ 0],
       [65],
       [36],
       [34],
       [64],
       [26],
       [19]])>

In [61]:
sampled_indices = tf.random.categorical(example_batch_predictions, num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()
sampled_indices


array([60, 43, 57, 21, 60, 34,  1, 21,  1, 39, 28, 42, 51, 41, 47, 34, 12,
       15, 65, 16, 30, 45, 65, 25, 45, 19,  5, 38, 39, 11, 30, 24, 10, 52,
       27,  3, 45, 46,  8, 41, 63, 22, 18, 44, 62, 25, 47,  0,  1, 33,  2,
       54,  8, 40, 55, 11, 43,  7, 50,  4, 62, 32, 46, 58])

In [49]:
len(sampled_indices)

64

In [26]:
target_example_batch

<tf.Tensor: shape=(64, 66), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>

In [60]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'Now sways it this way, like a mighty sea\nForced by the tide to combat with the wind;\nNow sways it th'

Next Char Predictions:
 b'EC$cdkDeo3:mGVulFCXM3z$OgqJxY[UNK]!Xe&k3-BGTHHeS3!?Pge;cAufy-Ajo:iBJ'


In [62]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'Now sways it this way, like a mighty sea\nForced by the tide to combat with the wind;\nNow sways it th'

Next Char Predictions:
 b'udrHuU\nH\nZOclbhU;BzCQfzLfF&YZ:QK3mN!fg-bxIEewLh[UNK]\nT o-ap:d,k$wSgs'


In [64]:
for input_example_batch, target_example_batch in dataset.take(1):
    print(input_example_batch)

tf.Tensor(
[[44 59  7 ... 43  2 40]
 [47 64  2 ... 57  2 58]
 [13  1  1 ... 59 54 57]
 ...
 [44  2 62 ... 44 59  2]
 [ 2 40 46 ... 47 54 60]
 [48 59 47 ... 47 44  2]], shape=(64, 100), dtype=int64)


In [68]:
start

<tf.Tensor: shape=(100,), dtype=int64, numpy=
array([44, 59,  7,  2, 58, 62, 44, 44, 59,  2, 53, 60, 57, 58, 44,  7,  2,
       59, 44, 51, 51,  2, 52, 44,  7,  2, 62, 47, 40, 59,  2, 58, 40, 64,
       58,  2, 52, 64,  2, 51, 54, 61, 44, 13,  1,  1, 27, 60, 57, 58, 44,
       11,  1, 38, 54, 60, 57,  2, 51, 54, 61, 44,  2, 58, 40, 64, 58,  7,
        2, 51, 48, 50, 44,  2, 40, 53,  2, 47, 54, 53, 44, 58, 59,  2, 46,
       44, 53, 59, 51, 44, 52, 40, 53,  7,  2, 40, 53, 43,  2, 40])>

In [339]:
start = input_example_batch[3]
start = np.reshape(start, (1, len(start), 1))
# # generate characters
# for i in range(1000):
#     x = np.reshape(start, (1, len(start), 1))

In [340]:
prediction = model2.predict(start)
index = np.argmax(prediction)#, num_samples = 1)


1/1 [==============================] - 0s 400ms/step


In [341]:
index

2

In [342]:
result = chars_from_ids(index).numpy()
list_seq = start.flatten()
list_seq = np.append(list_seq,index)
list_seq


array([ 3,  1, 33, 64, 41, 40, 51, 59,  7,  2, 26, 44, 57, 42, 60, 59, 48,
       54,  7,  2, 59, 47, 44,  2, 55, 57, 48, 53, 42, 44,  2, 44, 63, 55,
       57, 44, 58, 58, 51, 64,  2, 47, 40, 59, 47,  1, 19, 54, 57, 41, 48,
       43, 43, 44, 53,  2, 41, 40, 53, 43, 64, 48, 53, 46,  2, 48, 53,  2,
       35, 44, 57, 54, 53, 40,  2, 58, 59, 57, 44, 44, 59, 58, 11,  1, 21,
       54, 51, 43,  7,  2, 33, 64, 41, 40, 51, 59,  3,  2, 46, 54,  2])

In [343]:
for i in range(1000):
    list_seq = start.flatten()

    start = start[:,-100:,:]
    start = np.reshape(start, (1, len(start[0]), 1))
    prediction = model2.predict(start)
    # index = tf.random.categorical(prediction, num_samples = 1)
    index = np.argmax(prediction)
    list_seq = np.append(list_seq, index)
    start = np.reshape(list_seq, (1, len(list_seq), 1))


1/1 [==============================] - 0s 73ms/step


In [294]:
for i in range(1000):
    list_seq = start.flatten()

    start = start[:,-100:,:]
    start = np.reshape(start, (1, len(start[0]), 1))
    prediction = model2.predict(start)
    # index = tf.random.categorical(prediction, num_samples = 1)
    index = np.argmax(prediction)
    list_seq = np.append(list_seq, index)
    start = np.reshape(list_seq, (1, len(list_seq), 1))


1/1 [==============================] - 0s 74ms/step


In [258]:
print(tf.strings.join(chars_from_ids(list_seq)).numpy().decode('utf-8'))

 with a largess. Take your paper too,
And let me have them very well perfumed
For she is sweeter than and upponder'th
Oree, a mould alace!
But death y he breate is troo hast be buch they brettle tay thee tould they the the

Fist Carithere is troon, hid all mase to and fould
not to be out welcomes for and speed triath thou,
Lith the be a mouth reseed the hing ad we with your precesserverty
That where the here hat for his crace.
Mare to your great you bree will loods are of him,
With the were the hath here rate in my treed thou hast and cance of wind.

RUTESTIO:
I host but o's breathere bl ne more ence of War.

MENENIUS:
Of y undord is conder to sither,
To the protite y hear the e virguin the virguin of hearm minds so sigr.

KING RICHARD II:
The sigh rieser she warree to your grace.
Mare we was the be a mouth yree have be buct
To the prothere bland of the verooods
or prithere is arm the vingeding plote.
On elald of swe dood to Inder the gaids

METEUTIO:
A hast be a moutherer blands, it t

In [296]:
print(tf.strings.join(chars_from_ids(input_example_batch[3])).numpy().decode('utf-8'))

 with a largess. Take your paper too,
And let me have them very well perfumed
For she is sweeter tha


In [295]:
print(tf.strings.join(chars_from_ids(list_seq)).numpy().decode('utf-8'))

 with a largess. Take your paper too,
And let me have them very well perfumed
For she is sweeter thand of where is are hands,
And blood aplood as this inle be to and fare.

VUKE VINCENTIO:
Say, not so so the gool offirg instrime.

IO:
But thou like that she it you sorr to and for
I wor the ken she plowds thou conde to your gread of the daud,
And blood as thou losest thou conce stren where is the hath
And bllight vichall be Vurden,
And thou good and mout thou look should be to her prease of wan.

DiRENCELIO:
Hath not thou looks of your place.
MERCALUS:
That I whill he ast make the partited thou lott the ken she rast,
On elford and me come to mere buring.

Mard thee tay the oppartter.

Buting, With the cooding foodion;
And well he preade it the hand of whe calls minele,
In plood and me good and ape me to give you wer.

KING RICHARD IIIIII:
I hould be ke to mart thee takeses
And reade is no man my fore,
With him us no gour to geat my hadder,
And blood allook ho more ease to thee to theeTt

In [464]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Embedding

model4 = Sequential()
model4.add(Embedding(vocab_size, 
                    embedding_dim, input_length = 100, batch_input_shape=(64, 100)))
model4.add(GRU(rnn_units, stateful = True))
model4.add(Dense(vocab_size, activation = 'softmax'))

model4.summary()

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_17 (Embedding)    (64, 100, 256)            16896     
                                                                 
 gru_16 (GRU)                (64, 1024)                3938304   
                                                                 
 dense_7 (Dense)             (64, 66)                  67650     
                                                                 
Total params: 4,022,850
Trainable params: 4,022,850
Non-trainable params: 0
_________________________________________________________________


In [465]:
model4.compile(optimizer='adam',loss = 'categorical_crossentropy')#, loss=loss)

In [466]:
history4 = model4.fit(dataset, epochs=1)#, callbacks=[checkpoint_callback])

172/172 [==============================] - 224s 1s/step - loss: 2.9529


In [468]:
start.shape

(1, 100, 1)

In [471]:
model4.predict(start.reshape(1,100,1), batch_size = 1)

ValueError: in user code:

    File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/keras/engine/training.py", line 1845, in predict_function  *
        return step_function(self, iterator)
    File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/keras/engine/training.py", line 1834, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/keras/engine/training.py", line 1823, in run_step  **
        outputs = model.predict_step(data)
    File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/keras/engine/training.py", line 1791, in predict_step
        return self(x, training=False)
    File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) fro

In [348]:
start = input_example_batch[3]
start = np.reshape(start, (1, len(start), 1))
# # generate characters
# for i in range(1000):
#     x = np.reshape(start, (1, len(start), 1))

In [306]:
for input_example, target_example in dataset.take(1):
    print(input_example.shape)
    print(target_example[-2:])

(64, 100)
tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(2, 66), dtype=float32)


In [319]:
start.shape

(1, 100, 1)

In [320]:
prediction = model3.predict(start)
index = np.argmax(prediction)#, num_samples = 1)


InvalidArgumentError: Graph execution error:

Specified a list with shape [64,256] from a tensor with shape [1,256]
	 [[{{node TensorArrayUnstack/TensorListFromTensor}}]]
	 [[sequential_14/gru_10/PartitionedCall]] [Op:__inference_predict_function_254364]

In [321]:
for i in range(1000):
    list_seq = start.flatten()

    start = start[:,-100:,:]
    start = np.reshape(start, (1, len(start[0]), 1))
    prediction = model3.predict(start)
    # index = tf.random.categorical(prediction, num_samples = 1)
    index = np.argmax(prediction)
    list_seq = np.append(list_seq, index)
    start = np.reshape(list_seq, (1, len(list_seq), 1))

InvalidArgumentError: Graph execution error:

Specified a list with shape [64,256] from a tensor with shape [1,256]
	 [[{{node TensorArrayUnstack/TensorListFromTensor}}]]
	 [[sequential_14/gru_10/PartitionedCall]] [Op:__inference_predict_function_254364]

In [323]:
start.shape

(1, 100, 1)

In [328]:
input_example_batch

<tf.Tensor: shape=(64, 100), dtype=int64, numpy=
array([[44, 59,  7, ..., 43,  2, 40],
       [47, 64,  2, ..., 57,  2, 58],
       [13,  1,  1, ..., 59, 54, 57],
       ...,
       [44,  2, 62, ..., 44, 59,  2],
       [ 2, 40, 46, ..., 47, 54, 60],
       [48, 59, 47, ..., 47, 44,  2]])>

In [327]:
model3.predict(input_example_batch)

InvalidArgumentError: Graph execution error:

Specified a list with shape [64,256] from a tensor with shape [32,256]
	 [[{{node TensorArrayUnstack/TensorListFromTensor}}]]
	 [[sequential_14/gru_10/PartitionedCall]] [Op:__inference_predict_function_254926]

In [329]:
for input_example_batch, target_example_batch in dataset.take(1):
    print(input_example_batch)

tf.Tensor(
[[21 14 31 ... 58 59 40]
 [44 53 42 ...  2 40 53]
 [ 2 52 64 ... 40 57 50]
 ...
 [32 11  1 ... 34 32 11]
 [ 2 22 22 ... 47 59  2]
 [40 61 44 ... 54 53 44]], shape=(64, 100), dtype=int64)


In [333]:
model3.predict(input_example_batch[0,:])

InvalidArgumentError: Graph execution error:

transpose expects a vector of size 2. But input(1) is a vector of size 3
	 [[{{node transpose}}]]
	 [[sequential_14/gru_10/PartitionedCall]] [Op:__inference_predict_function_254926]

In [344]:
model2.input_shape

(None, 100)

In [423]:
for input_example_batch, target_example_batch in dataset.take(1):
    print(input_example_batch.shape)

(64, 100)


In [349]:
model2.predict(start)

1/1 [==============================] - 0s 112ms/step


array([[5.89543597e-05, 4.51959744e-02, 1.36264846e-01, 1.22864638e-03,
        1.10425004e-04, 5.90695054e-05, 8.00059922e-03, 2.58162636e-02,
        9.28002119e-04, 1.43156061e-03, 4.69483966e-05, 5.20989895e-01,
        4.04408388e-03, 1.67755899e-03, 7.38662900e-03, 7.78966874e-04,
        2.06340440e-02, 2.51363660e-03, 1.76093057e-02, 6.05375506e-04,
        8.36621271e-04, 1.86338299e-03, 1.59986429e-02, 1.65830017e-04,
        3.46939400e-04, 8.69356375e-03, 1.20941759e-03, 7.02577308e-02,
        3.49683501e-02, 3.38843733e-04, 1.13477436e-04, 1.53632415e-02,
        1.30405873e-02, 1.16558336e-02, 9.01478343e-03, 1.42443948e-03,
        1.98968314e-03, 1.05877967e-04, 3.86483589e-04, 2.36809836e-04,
        8.85868561e-04, 2.24677511e-04, 2.03902353e-04, 7.13696587e-04,
        1.62163307e-03, 3.45456996e-04, 2.50571320e-04, 2.30336052e-04,
        1.08035852e-03, 1.29702996e-04, 1.64061770e-04, 1.43042114e-03,
        1.80593473e-04, 1.99430995e-03, 7.30948930e-04, 7.207976

In [389]:
model2.predict(np.reshape(input_example_batch[0], (1, 100, 1)))

1/1 [==============================] - 0s 87ms/step


array([[5.3781982e-06, 1.2437294e-03, 1.2833575e-02, 1.6953218e-04,
        4.4371432e-06, 4.7109070e-06, 3.4420996e-03, 2.6707505e-03,
        1.5192543e-04, 9.7222533e-04, 4.9438577e-06, 1.4715543e-04,
        2.7260219e-04, 2.8289243e-04, 1.4059538e-04, 9.8925069e-05,
        2.6460808e-05, 1.8025046e-05, 5.3791449e-05, 2.1343671e-05,
        5.8500093e-05, 3.1138905e-05, 3.0022985e-04, 1.2000801e-05,
        3.4579567e-05, 3.1888743e-05, 1.6583199e-05, 4.1761949e-05,
        7.5731026e-05, 1.4172477e-05, 8.2835813e-06, 7.5529650e-05,
        3.4264853e-05, 1.2708161e-04, 7.4096046e-05, 1.5994844e-04,
        6.9371403e-05, 7.8206967e-06, 2.4674046e-05, 8.6217397e-06,
        3.4291202e-01, 2.3365160e-04, 3.1399369e-04, 2.6283038e-04,
        3.2726511e-01, 3.9598113e-03, 4.5833993e-04, 1.0682629e-02,
        7.1162082e-02, 3.4446133e-05, 3.2565222e-04, 7.0882938e-03,
        7.8178372e-04, 2.4569589e-03, 1.5802439e-01, 1.0855151e-03,
        4.1681531e-05, 1.7436078e-02, 2.9756124e

In [455]:
model3.predict(input_example_batch,batch_size=64).shape

1/1 [==============================] - 0s 254ms/step


(64, 66)

In [403]:
model2.input_shape

(None, 100)

In [408]:
model2.predict(start)

1/1 [==============================] - 0s 125ms/step


array([[5.89543597e-05, 4.51959744e-02, 1.36264846e-01, 1.22864638e-03,
        1.10425004e-04, 5.90695054e-05, 8.00059922e-03, 2.58162636e-02,
        9.28002119e-04, 1.43156061e-03, 4.69483966e-05, 5.20989895e-01,
        4.04408388e-03, 1.67755899e-03, 7.38662900e-03, 7.78966874e-04,
        2.06340440e-02, 2.51363660e-03, 1.76093057e-02, 6.05375506e-04,
        8.36621271e-04, 1.86338299e-03, 1.59986429e-02, 1.65830017e-04,
        3.46939400e-04, 8.69356375e-03, 1.20941759e-03, 7.02577308e-02,
        3.49683501e-02, 3.38843733e-04, 1.13477436e-04, 1.53632415e-02,
        1.30405873e-02, 1.16558336e-02, 9.01478343e-03, 1.42443948e-03,
        1.98968314e-03, 1.05877967e-04, 3.86483589e-04, 2.36809836e-04,
        8.85868561e-04, 2.24677511e-04, 2.03902353e-04, 7.13696587e-04,
        1.62163307e-03, 3.45456996e-04, 2.50571320e-04, 2.30336052e-04,
        1.08035852e-03, 1.29702996e-04, 1.64061770e-04, 1.43042114e-03,
        1.80593473e-04, 1.99430995e-03, 7.30948930e-04, 7.207976

In [404]:
input_example_batch[0].shape

TensorShape([100])

In [405]:
start.shape

(1, 100, 1)

In [406]:
type(start)

numpy.ndarray

In [407]:
type(input_example_batch[0])

tensorflow.python.framework.ops.EagerTensor

In [409]:
model3.input_shape

(64, 100)

In [438]:
model3.predict(np.reshape(input_example_batch, (64, 100, 1)))

InvalidArgumentError: Graph execution error:

Specified a list with shape [64,256] from a tensor with shape [32,256]
	 [[{{node TensorArrayUnstack/TensorListFromTensor}}]]
	 [[sequential_14/gru_10/PartitionedCall]] [Op:__inference_predict_function_254364]

In [417]:
for i in range(1):
    list_seq = start.flatten()

    start = start[:,-100:,:]
    print(start.shape)

    start = np.reshape(start, (1, len(start[0]), 1))
    print(start.shape)
    # prediction = model2.predict(start)
    # # index = tf.random.categorical(prediction, num_samples = 1)
    # index = np.argmax(prediction)
    # list_seq = np.append(list_seq, index)
    # start = np.reshape(list_seq, (1, len(list_seq), 1))

(1, 100, 1)
(1, 100, 1)


In [442]:
np.reshape(input_example_batch, (64, 100, 1)).shape

(64, 100, 1)

In [513]:
model5(start, return_state = True)

TypeError: call() got an unexpected keyword argument 'return_state'

In [511]:
model5(start)

<tf.Tensor: shape=(1, 66), dtype=float32, numpy=
array([[3.12957910e-10, 1.70277765e-08, 1.18758835e-04, 1.80441617e-07,
        3.96180699e-10, 1.68964165e-09, 2.44642173e-08, 6.10254673e-08,
        4.43373516e-08, 6.39324327e-08, 3.29681782e-10, 1.01852384e-08,
        3.67712943e-10, 3.62134500e-09, 2.31547901e-08, 3.80895386e-08,
        1.11368206e-06, 1.91714733e-09, 2.50724619e-09, 6.29242658e-10,
        2.11106030e-08, 8.63253717e-08, 1.45852136e-10, 8.39600531e-08,
        4.59556198e-08, 4.13149763e-08, 4.34148220e-08, 5.97581845e-07,
        7.30372873e-10, 2.11899529e-08, 6.41865117e-10, 6.92192303e-09,
        2.31350583e-08, 5.78950976e-09, 1.99836014e-09, 1.35016665e-08,
        4.75396433e-08, 1.47091566e-08, 3.31229311e-09, 2.17056106e-09,
        1.75584233e-07, 3.94039887e-07, 9.89628679e-05, 1.76704589e-06,
        1.21203038e-06, 8.88815464e-07, 1.62587287e-06, 5.64799492e-11,
        2.08148731e-06, 3.70458828e-08, 1.27576982e-07, 3.56784803e-05,
        1.15548

In [478]:
start.shape

(1, 100, 1)

In [525]:
start = input_example_batch[4]
start = np.reshape(start, (1, len(start), 1))
# # generate characters
# for i in range(1000):
#     x = np.reshape(start, (1, len(start), 1))

In [498]:
for i in range(1000):
    list_seq = start.flatten()

    start = start[:,-100:,:]
    start = np.reshape(start, (1, len(start[0]), 1))
    prediction = model5.predict(start)
    # index = tf.random.categorical(prediction, num_samples = 1)
    index = np.argmax(prediction)
    list_seq = np.append(list_seq, index)
    start = np.reshape(list_seq, (1, len(list_seq), 1))


1/1 [==============================] - 0s 71ms/step


In [479]:
for i in range(1000):
    list_seq = start.flatten()
    
    start = start[:,-100:,:]
    start = np.reshape(start, (1, len(start[0]), 1))
    prediction = model5.predict(start)
    # index = tf.random.categorical(prediction, num_samples = 1)
    index = np.argmax(prediction)
    list_seq = np.append(list_seq, index)
    start = np.reshape(list_seq, (1, len(list_seq), 1))


1/1 [==============================] - 0s 67ms/step


In [480]:
print(tf.strings.join(chars_from_ids(list_seq)).numpy().decode('utf-8'))

 you a better husband.

MARIANA:
O my dear lord,
I crave no other, nor no better man.

DUKE VINCENTIO:
I hash not it be not the king,
Which rother it y hand not she ken she wence vince.
Whot she in the ver on the past more have be of Heart,
To good and thou love the thould cove with him bland the vence of Whis preade,
And blood ason, the ware to andereffer the
bost and ret my lord,--hith bland the wastreeterss of his dound,
Which rother is the hands of hear me a mandsem,
And blake a mouther, boy thou last and lets fime,
Which ence of Whis now in the very,
And to go so ve to make the very hor thoughto-

JUCESTEO:

HORD:
He hands be be bucttett to and ferey thee?

CORUCESTEO:
Sale, and strince's the sweed thing?

MERCANTIO:
Sall ne so sin, sie hather blot!

KING RICHARD II:
I bo your gordent of Vourte;
Ond gargen and out fall be buch be treed
That is a mand the had he pure to and ferey
Stoon and all he pure to and fere the ears of hear.

METEN:
Of farthem, boyour agacestercH.
ANd thou ga

In [517]:
print(tf.strings.join(chars_from_ids(input_example_batch[4])).numpy().decode('utf-8'))

him at our pleasure?
Our scouts have found the adventure very easy:
That as Ulysses and stout Diomed


In [500]:
print(tf.strings.join(chars_from_ids(list_seq)).numpy().decode('utf-8'))

hat's the matter?

Messenger:
You are sent for to the Capitol. 'Tis thought
That Marcius shall be condent to thee to-
And the well we dake of Burrentertseing foresessss,
And samfices frew he bant to sinnes
I
will lake the plope to the peoply mereere to andereft

JUCESTEO:
Sall ne more,--
Ordengion the wholl comfund this isllace.

Wist Cotine :
I blang of E mand the well we dave us mine
That think he a thou dound the hear for the eath the pentuetettate;
And Glove a mand of bloud to thee takes of wist thoull.

CORUCENOUCESTIO:
Sall ne mare be say thee takes of wer.

Deat I will loke, thou lovest I'll wed.

Fisst Martee:
I hor her so ser and aploud alate!
Oy the swear'd and it is amand the hands of him be suge,
And blaigh of blonds, sither, Ind and my dow, in the virg,
What now be was the benou have us no gord and the vencestovest on thy douth
And blaty do you be a mand of for theest.

ClAUCESTEN:
No, not ke warre to sarre?

IUSENLE:
Why she sat's your grrance My combond!

MARCANI IV AUd 

In [521]:
for i in range(1000):
    list_seq = start.flatten()
    model5.reset_states()
    start = start[:,-100:,:]
    start = np.reshape(start, (1, len(start[0]), 1))
    prediction = model5.predict(start)
    # index = tf.random.categorical(prediction, num_samples = 1)
    index = np.argmax(prediction)
    list_seq = np.append(list_seq, index)
    start = np.reshape(list_seq, (1, len(list_seq), 1))


1/1 [==============================] - 0s 68ms/step


In [522]:
print(tf.strings.join(chars_from_ids(list_seq)).numpy().decode('utf-8'))

him at our pleasure?
Our scouts have found the adventure very easy:
That as Ulysses and stout Diomedeet
Buchtent to thee to sirne,
Which rother is the hand of hear fee how be suge tan
That my head me browniding greene ronner'st:
Pratier and ret his lake the hear for the strince.

Whow not the ken, of homours,
Yer farter so singes, they sass ciming forsenss whe son,
And love I math blene of Hambugeding;
Sicht and me so sind free hor he a mantere the
bood and my downing is ard net make the bather
How by the know but He ruther
Hin for her suges, with the cander to serre.

I wist know, ard condert
It for hears aplicells forese:
I wnot not and let me be uld and the hands of him strends,
But thou hast so sin, sie we to sille If y uptrince.

Wist to Gome of mear, to time
Which ence of Whis count wo thees
It place un one rase is arding the swerd,
Which rothering is crisins for leess of hears, sinder
This plane the wening and tee hath be nught ond frings,
But thou loss thou lass the hand of ble

In [526]:
for i in range(1000):
    list_seq = start.flatten()
    # model5.reset_states()
    start = start[:,-100:,:]
    start = np.reshape(start, (1, len(start[0]), 1))
    prediction = model5.predict(start)
    # index = tf.random.categorical(prediction, num_samples = 1)
    index = np.argmax(prediction)
    list_seq = np.append(list_seq, index)
    start = np.reshape(list_seq, (1, len(list_seq), 1))


1/1 [==============================] - 0s 96ms/step


In [527]:
print(tf.strings.join(chars_from_ids(list_seq)).numpy().decode('utf-8'))

him at our pleasure?
Our scouts have found the adventure very easy:
That as Ulysses and stout Diomedeet
Buchtent to thee to me reed thing it they hadd theee.

CORINGO:
But thou last mo the couds of wind the vence.
Whot now the her she leess y ur enee we ard thee hat for hears for heess speed free
To Berrentering preccecusted thee?

JULEET:
A, shall we do a more ris conding thee to sign;
And she plopes ereere I am net made be suge,
And blace, and stee fortherst indore the war.

I GoLIO:
As no a growniding to be guee to thee?

Mistreen's mandere:
I bo a bood and me bood the ware to andefte;
And Glove a mand of bloud to thee takes of wist thould congents
Or pracesser for thee tansements
of blance, so singe
To and ret s y ulloud as the bation; lord,
What day you be a mand of bland
Dounder to gie we the beould of hor moust:
Thit had be a mind a mand the had he pure to and ferey
Stoon and all he purine meen, ard thee hath be not thee take,
What you y up y his he breath in the verver's of whe